In [3]:
#Connect to mySQL database
import mysql.connector
mydb = mysql.connector.connect(
host="127.0.0.1",
port=3306,
user="root",
password="root")
print(mydb)

mycursor = mydb.cursor(buffered=True)

In [4]:
#Create a Database named 110304015 and update the cursor
mycursor.execute("CREATE DATABASE Tiny_project")

mydb = mysql.connector.connect(
host="127.0.0.1",
port=3306,
user="root",
password="root",
database="Tiny_project")

mycursor = mydb.cursor(buffered=True)

In [5]:
#Create Tables
mycursor.execute('CREATE TABLE photos (species VARCHAR(255), id INT)')
mycursor.execute('CREATE TABLE metadata (id INT,GPSInfo INT, ResolutionUnit INT, ExifOffset INT,Type INT,Orientation INT,DateTime TIMESTAMP,YCbCrPositioning INT,YResolution INT,Copyright VARCHAR(255),XResolution INT,Artist VARCHAR(255))')
mycursor.execute('CREATE TABLE model_type (Type INT,Make VARCHAR(255), Model VARCHAR(255),Software VARCHAR(255))')
#Add PK and INDEX, Create Relations
mycursor.execute("ALTER TABLE model_type ADD PRIMARY KEY (Type)")
#mycursor.execute("ALTER TABLE metadata ADD FORIEGN KEY (Make) REFERENCES")
mycursor.execute('ALTER TABLE metadata ADD CONSTRAINT FOREIGN KEY (Type) references model_type (Type)')
mycursor.execute("SET GLOBAL FOREIGN_KEY_CHECKS=0")

In [6]:
#Insert the type of models into model_type table
type_of_models = []
sql = "INSERT INTO model_type (Type, Make, Model, Software) VALUES (%s, %s, %s, %s)"
val = (0,'Canon','Canon EOS R7', 'Digital Photo Professional')
type_of_models.append(val)
mycursor.execute(sql,val)
sql = "INSERT INTO model_type (Type, Make, Model, Software) VALUES (%s, %s, %s, %s)"
val = (1,'Canon','Canon EOS R', 'Digital Photo Professional')
mycursor.execute(sql,val)
type_of_models.append(val)

In [7]:
#define a function to conveniently save images

from PIL import Image
from PIL.ExifTags import TAGS
def save_img(filename,species,index):
    #For photos table
    img = open(filename,'rb')
    sql = "INSERT INTO photos (species, id) VALUES (%s, %s)"
    val = (species,index)
    mycursor.execute(sql, val)#populate_data1
    mydb.commit()
    #For metadata table
    image = Image.open(filename)
    exifdata = image.getexif()
    values = []
    for tag_id in exifdata:
    # get the tag name, instead of human unreadable tag id
        tag = TAGS.get(tag_id, tag_id)
        data = exifdata.get(tag_id)
        if isinstance(data, bytes):
            data = data.decode()
        values.append(data)
    type_ = 0
    temp_ind = 0
    for types in type_of_models:
        if( (values[3],values[4],values[5]) == (types[1],types[2],types[3]) ):
            type_ = types[0]
    sql_meta = "INSERT INTO metadata (id,GPSInfo,ResolutionUnit,ExifOffset,Type,Orientation,DateTime,YCbCrPositioning,YResolution,Copyright,XResolution,Artist) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    val_meta = (index,values[0],values[1],values[2],type_,values[6],values[7],values[8],int(values[9]),values[10],int(values[11]),values[12])
    mycursor.execute(sql_meta, val_meta)
    mydb.commit()

In [13]:
#Setting directory of images
import os 
dir_path_1 = os.path.dirname(os.path.realpath('/Users/wuguanxun/Desktop/photos/bird_photos/Pycnonotus_sinensis/6B1A1285.JPG'))
file_names_1 = os.listdir(dir_path_1)

dir_path_2 = os.path.dirname(os.path.realpath('/Users/wuguanxun/Desktop/photos/bird_photos/Psilopogon_nuchalis/6B1A1109.JPG'))
file_names_2 = os.listdir(dir_path_2)

In [14]:
#Saving images
index = 0
file_names_1.remove('.DS_Store')
for file in file_names_1:
    save_img("/Users/wuguanxun/Desktop/photos/bird_photos/Pycnonotus_sinensis/"+file,'Pycnonotus sinensis',index)
    index = index+1
for file in file_names_2:
    save_img("/Users/wuguanxun/Desktop/photos/bird_photos/Psilopogon_nuchalis/"+file,'Psilopogon nuchalis',index)
    index = index+1